# Regression Project: Predicting the Sale Prices of Homes in Seattle, Washington

Dataset is downloaded from Kaggle: https://www.kaggle.com/datasets/samuelcortinhas/house-price-prediction-seattle/code

In [37]:
# Importing the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20, 10)


In [38]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [39]:
df_train.head()

,beds,baths,size,size_units,lot_size,lot_size_units,zip_code,price
0,3,2.5,2590.0,sqft,6000.00,sqft,98144,795000.0
1,4,2.0,2240.0,sqft,0.31,acre,98106,915000.0
2,4,3.0,2040.0,sqft,3783.00,sqft,98107,950000.0
3,4,3.0,3800.0,sqft,5175.00,sqft,98199,1950000.0
4,2,2.0,1042.0,sqft,NaN,NaN,98102,950000.0


In [40]:
df_train.shape, df_test.shape

((2016, 8), (505, 8))

In [41]:
df_train.info(), df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   beds            2016 non-null   int64  
 1   baths           2016 non-null   float64
 2   size            2016 non-null   float64
 3   size_units      2016 non-null   object 
 4   lot_size        1669 non-null   float64
 5   lot_size_units  1669 non-null   object 
 6   zip_code        2016 non-null   int64  
 7   price           2016 non-null   float64
dtypes: float64(4), int64(2), object(2)
memory usage: 126.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   beds            505 non-null    int64  
 1   baths           505 non-null    float64
 2   size            505 non-null    float64
 3   size_units      505 non-null    object 
 4   l

(None, None)

## Exploring the dataset

Note that:
- beds = number of bedrooms in the house
- baths = number of bathrooms in the house and 0.5 means a half-bathroom which has a sink and a toilet but no shower or bathtub
- size = the size of the house in square feet
- size_units = the unit of the size of the house
- lot_size = the total area of the land where the property is located on. The lot belongs to the house owner.
- lot_size_units = the unit of the lot size
- zip_code = the zip code of the location of the house
- price = the price of the house in dollars (USD)
- 1 acre = 43560 square feet


In [42]:
# Checking for missing values
df_train.isnull().sum(), df_test.isnull().sum()

(beds                0
 baths               0
 size                0
 size_units          0
 lot_size          347
 lot_size_units    347
 zip_code            0
 price               0
 dtype: int64,
 beds               0
 baths              0
 size               0
 size_units         0
 lot_size          77
 lot_size_units    77
 zip_code           0
 price              0
 dtype: int64)

In [43]:
# Checking for the proportion of missing values in the dataset
df_train.isnull().sum()/df_train.shape[0]*100, df_test.isnull().sum()/df_test.shape[0]*100

(beds               0.000000
 baths              0.000000
 size               0.000000
 size_units         0.000000
 lot_size          17.212302
 lot_size_units    17.212302
 zip_code           0.000000
 price              0.000000
 dtype: float64,
 beds               0.000000
 baths              0.000000
 size               0.000000
 size_units         0.000000
 lot_size          15.247525
 lot_size_units    15.247525
 zip_code           0.000000
 price              0.000000
 dtype: float64)

Huge percentage of missing values in the lot_size column as well as the corresponding lot_size_units column in both datasets. We will assume that the minimum lot size  will be the size_unit, which is what we will use to fill it. This helps to logically ensure that the lot size is not less than the size of the house and deviation from the truth is minimal.

In [44]:
df_train[['size', 'size_units', 'lot_size', 'lot_size_units']].head()

,size,size_units,lot_size,lot_size_units
0,2590.0,sqft,6000.00,sqft
1,2240.0,sqft,0.31,acre
2,2040.0,sqft,3783.00,sqft
3,3800.0,sqft,5175.00,sqft
4,1042.0,sqft,NaN,NaN


In [45]:
df_train.groupby('lot_size_units')['lot_size_units'].count()

lot_size_units
acre     220
sqft    1449
Name: lot_size_units, dtype: int64

In [46]:
df_test.groupby('lot_size_units')['lot_size_units'].count()

lot_size_units
acre     59
sqft    369
Name: lot_size_units, dtype: int64

## Data Cleaning and Preprocessing

In [47]:
# Some values in the lot_size_units column are in acres and some are in sqft. We will convert all the values to sqft

# We will define a function to convert the values in the lot_size column to sqft
def convert_to_sqft(row):
    if row['lot_size_units'] == 'sqft':
        return row['lot_size']
    else:
        return row['lot_size']*43560

In [48]:
# Applying the function to the dataset
df_train['lot_size'] = df_train.apply(lambda row: convert_to_sqft(row), axis=1)

In [49]:
df_train.head()

,beds,baths,size,size_units,lot_size,lot_size_units,zip_code,price
0,3,2.5,2590.0,sqft,6000.0,sqft,98144,795000.0
1,4,2.0,2240.0,sqft,13503.6,acre,98106,915000.0
2,4,3.0,2040.0,sqft,3783.0,sqft,98107,950000.0
3,4,3.0,3800.0,sqft,5175.0,sqft,98199,1950000.0
4,2,2.0,1042.0,sqft,NaN,NaN,98102,950000.0


In [50]:
df_test['lot_size'] = df_test.apply(lambda row: convert_to_sqft(row), axis=1)

In [51]:
df_test.head()

,beds,baths,size,size_units,lot_size,lot_size_units,zip_code,price
0,3,3.0,2850.0,sqft,4200.0,sqft,98119,1175000.0
1,4,5.0,3040.0,sqft,5002.0,sqft,98106,1057500.0
2,3,1.0,1290.0,sqft,6048.0,sqft,98125,799000.0
3,3,2.0,2360.0,sqft,12196.8,acre,98188,565000.0
4,3,3.5,1942.0,sqft,1603.0,sqft,98107,1187000.0


In [52]:
# Changing the lot_size_units column to sqft
df_train['lot_size_units'] = 'sqft'
df_test['lot_size_units'] = 'sqft'


In [53]:
df_train.head()

,beds,baths,size,size_units,lot_size,lot_size_units,zip_code,price
0,3,2.5,2590.0,sqft,6000.0,sqft,98144,795000.0
1,4,2.0,2240.0,sqft,13503.6,sqft,98106,915000.0
2,4,3.0,2040.0,sqft,3783.0,sqft,98107,950000.0
3,4,3.0,3800.0,sqft,5175.0,sqft,98199,1950000.0
4,2,2.0,1042.0,sqft,NaN,sqft,98102,950000.0


In [54]:
# Dealing with missing values

#1. Finding the average lot_size:size and applying it to the missing values
df_train['avg_lot_ratio'] = df_train['lot_size']/df_train['size']
df_train.head()

,beds,baths,size,size_units,lot_size,lot_size_units,zip_code,price,avg_lot_ratio
0,3,2.5,2590.0,sqft,6000.0,sqft,98144,795000.0,2.316602
1,4,2.0,2240.0,sqft,13503.6,sqft,98106,915000.0,6.028393
2,4,3.0,2040.0,sqft,3783.0,sqft,98107,950000.0,1.854412
3,4,3.0,3800.0,sqft,5175.0,sqft,98199,1950000.0,1.361842
4,2,2.0,1042.0,sqft,NaN,sqft,98102,950000.0,NaN


In [55]:
df_train['avg_lot_ratio'].mean()

38.00439044304304

In [56]:
df_train['avg_lot_ratio'].median()

2.334375

In [57]:
df_train['avg_lot_ratio'].max()

43560.0

The data above implies that the dataset is likely heavily skewed to the right (positively skewed) probably due the presence of outliers or extremely high values in the dataset

In [59]:
# Sorting and showing the biggest ten values

df_train.sort_values(by='avg_lot_ratio', na_position='first').tail(20)


,beds,baths,size,size_units,lot_size,lot_size_units,zip_code,price,avg_lot_ratio
645,2,2.5,1365.0,sqft,176418.0,sqft,98118,525000.0,129.243956
952,2,2.0,877.0,sqft,114562.8,sqft,98115,415000.0,130.630331
1394,2,2.0,1006.0,sqft,144619.2,sqft,98112,650000.0,143.756660
266,2,2.0,1245.0,sqft,179031.6,sqft,98168,330000.0,143.800482
1908,2,2.5,1138.0,sqft,172933.2,sqft,98126,515000.0,151.962390
1451,2,1.0,885.0,sqft,154202.4,sqft,98105,389000.0,174.240000
783,1,1.0,626.0,sqft,114562.8,sqft,98115,275000.0,183.007668
465,2,1.0,903.0,sqft,176853.6,sqft,98146,363500.0,195.851163
1218,2,1.0,903.0,sqft,176853.6,sqft,98146,312500.0,195.851163
596,2,2.0,1415.0,sqft,278784.0,sqft,98125,449000.0,197.020495


Lot size above 202989.6sqft are massive outliers and would not be used.

In [60]:
# Repalcing the outliers with NaN in the lot_size column

df_train['lot_size'] = df_train['lot_size'].where(df_train['avg_lot_ratio'] <= 290, np.nan)

df_train.sort_values(by='avg_lot_ratio', na_position='first').tail(20)

,beds,baths,size,size_units,lot_size,lot_size_units,zip_code,price,avg_lot_ratio
645,2,2.5,1365.0,sqft,176418.0,sqft,98118,525000.0,129.243956
952,2,2.0,877.0,sqft,114562.8,sqft,98115,415000.0,130.630331
1394,2,2.0,1006.0,sqft,144619.2,sqft,98112,650000.0,143.756660
266,2,2.0,1245.0,sqft,179031.6,sqft,98168,330000.0,143.800482
1908,2,2.5,1138.0,sqft,172933.2,sqft,98126,515000.0,151.962390
1451,2,1.0,885.0,sqft,154202.4,sqft,98105,389000.0,174.240000
783,1,1.0,626.0,sqft,114562.8,sqft,98115,275000.0,183.007668
465,2,1.0,903.0,sqft,176853.6,sqft,98146,363500.0,195.851163
1218,2,1.0,903.0,sqft,176853.6,sqft,98146,312500.0,195.851163
596,2,2.0,1415.0,sqft,278784.0,sqft,98125,449000.0,197.020495


In [61]:
# Counting the number of missing values in the lot_size column, it should be 2 more than the previous count

df_train['lot_size'].isnull().sum()

349